In [1]:
# CNN model for CIFAR-10
import numpy
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras.losses import mse, logcosh, binary_crossentropy, categorical_crossentropy
from keras.activations import relu, elu, softmax, sigmoid, linear
from keras.constraints import maxnorm
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras import Input, Model
from keras.layers import Dropout
from keras.layers import LSTM, Reshape
from keras.layers import TimeDistributed, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import Input, Model
from keras.callbacks import TensorBoard
from keras.layers import Dense, Add, Dropout
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import TensorBoard
import random

Using TensorFlow backend.


### Fonction de construction du ResNets

In [2]:
def network_builder(network_conf, network_archi):
    
    while len(network_conf) > 0:
        
        # Premiere couche
        if 'input_layer' in network_conf:
            
            # Add first layer
            input_layer = Input(shape=(network_conf['input_layer']['input_dim'],))
            
            # Add first hidden Layer
            hidden1 = Dense(network_conf['hidden_layer']['hidden_dim'],
                           activation=network_conf['hidden_layer']['activation'])(input_layer)
            
            hidden2 = Dense(network_conf['hidden_layer']['hidden_dim'],
                            activation=network_conf['hidden_layer']['activation'])(hidden1)
            
            res = Add()([hidden1, hidden2])
            
            #store data in list
            network_archi.append(input_layer)
            network_archi.append(hidden1)
            network_archi.append(hidden2)
            network_archi.append(res)
            
            
            # Delete pills of stack
            network_conf.pop('input_layer')
            network_builder(network_conf, network_archi)
            
        # Couche caché
        elif len(network_conf) > 1 and network_conf['hidden_layer']['nbr_hidden_layer'] > 0:
            
            hidden = Dense(network_conf['hidden_layer']['hidden_dim'], 
                           activation=network_conf['hidden_layer']['activation'])(network_archi[-1])
            
            res = Add()([network_archi[-2], hidden])
            
            network_archi.append(hidden)
            network_archi.append(res)
            
            # decrement nbr_hidden_layer
            network_conf['hidden_layer']['nbr_hidden_layer'] -= 1
            
            if(network_conf['hidden_layer']['nbr_hidden_layer'] == 0):
                # Derniere couche
                network_conf.pop('hidden_layer')
            
            network_builder(network_conf, network_archi)
        
        elif len(network_conf) == 1:
            output_layer = Dense(network_conf['output_layer']['output_dim'],
                                 activation=network_conf['output_layer']['activation'])(network_archi[-1])

            network_archi.append(output_layer)
            network_conf.pop('output_layer')
            
            network_archi.append(Model(network_archi[0], output_layer))

# I.Parametrage des hyper-paramètres

In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [4]:
lr = numpy.random.random() * (0.1 - 0.0001) + 0.0001
momentum = numpy.random.random() * (0.1 - 0.0001) + 0.0001
epochs = 1
decay = lr / epochs

In [17]:
Param = {'input_shape': 3072,
             'input_shape_rnn': (32, 96),
             'input_shape_cnn': (32, 32, 3),
             'lr': lr,
             'hidden_dim': 128,
             'units': 512,
             'unitsSlp': 10,
             'last_units': 10,
             'first_neuron': [4, 8, 16, 32, 64],
             'hidden_layers': [2, 4, 6, 8, 9, 10, 20, 25, 30],
             'kernel_constraint': maxnorm(3),
             'batch_size': (64, 128, 512, 1024, 2048),
             'epochs': epochs,
             'dropout': (0, 0.5, 1),
             'padding': 'same',
             'metrics': ['accuracy'],
             'weight_regulizer': [None],
             'emb_output_dims': [None],
             'shape': ['brick', 'long_funnel'],
             'optimizer': ['adam', 'Nadam', 'RMSprop', SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)],
             'losses': [mse, logcosh, binary_crossentropy, categorical_crossentropy],
             'activation': [relu, elu, linear],
             'last_activation': [softmax, sigmoid],
             'nb_classes': 10}

In [6]:
def _random_param(param):
    """
    _random_param : do random to param values
    :param param:
    :return:
    """
    rand_param = {'input_shape': param['input_shape'],
                  'input_shape_cnn': param['input_shape_cnn'],
                  'input_shape_rnn': param['input_shape_rnn'],
                  'lr': param['lr'],
                  'hidden_dim': param['hidden_dim'],
                  'units': param['units'],
                  'unitsSlp': param['unitsSlp'],
                  'padding': param['padding'],
                  'last_units': param['last_units'],
                  'first_neuron': random.choice(param['first_neuron']),
                  'hidden_layers': random.choice(param['hidden_layers']),
                  'kernel_constraint': param['kernel_constraint'],
                  'batch_size': random.choice(param['batch_size']),
                  'epochs': param['epochs'],
                  'dropout': random.choice(param['dropout']),
                  'metrics': param['metrics'],
                  'weight_regulizer': param['weight_regulizer'],
                  'emb_output_dims': ['emb_output_dims'],
                  'shape': random.choice(param['shape']),
                  'optimizer': random.choice(param['optimizer']),
                  'losses': random.choice(param['losses']),
                  'activation': random.choice(param['activation']),
                  'last_activation': random.choice(param['last_activation']),
                  'nb_classes': param['nb_classes']}
    return rand_param

# II. Prétraitement de la donnée

In [7]:
def _preprocess_cifar10(__param):
    """
    :param dataset:
    :return:
    """
    (__X_train, __y_train), (__X_test, __y_test) = cifar10.load_data()
    __X_train = __X_train.reshape(50000, 32 * 32 * 3)
    __X_test = __X_test.reshape(10000, 32 * 32 * 3)

    __X_train = __X_train.astype('float32')
    __X_test = __X_test.astype('float32')
    __X_train /= 255.0
    __X_test /= 255.0

    __y_train = np_utils.to_categorical(__y_train, __param['nb_classes'])
    __y_test = np_utils.to_categorical(__y_test, __param['nb_classes'])
    return (__X_train, __y_train), (__X_test, __y_test)

## 1. Charge et prépare le dataset

In [8]:
param = _random_param(Param)
dataset = _preprocess_cifar10(param)

In [9]:
# load data
(X_train, y_train), (X_test, y_test) = dataset
nb_classes = y_test.shape[1]

# III. Gestion du modèle

## 1. Paramètrage du modèle

In [10]:
def __network_architecture_builder(param):
    """
    network_architecture_builder : Configuration of architecture ResNets network
    :return:
    """

    network_conf = {}
    network_conf.update({'input_layer': {'input_dim': param['input_shape']}
                         })

    network_conf.update({'hidden_layer': {'hidden_dim': param['hidden_dim'],
                                          'activation': param['activation'],
                                          'nbr_hidden_layer': param['hidden_layers']
                                          }
                         })

    network_conf.update({'output_layer': {'output_dim': param['unitsSlp'],
                                          'activation': param['last_activation']}
                         })
    return network_conf

In [11]:
_network_config = __network_architecture_builder(param)
_network_architecture = []

## 2. Construction du modèles

In [12]:
# Call function to build network
network_builder(_network_config,_network_architecture)

# Call model
model = _network_architecture[-1]

## 3. compile le modèle

In [13]:
# Compile model
model.compile(loss=param['losses'],
              optimizer=param['optimizer'],
              metrics=param['metrics'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3072)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          393344      input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          16512       dense_1[0][0]                    
__________________________________________________________________________________________________
add_1 (Add)                     (None, 128)          0           dense_1[0][0]                    
                                                                 dense_2[0][0]                    
__________

## 4. Entrainement le modèle

In [14]:
# training
history = model.fit(X_train, y_train,
                    batch_size=param['batch_size'],
                    epochs=param['epochs'],
                    verbose=1,
                    validation_data=(X_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 4s 87us/step - loss: 1.7053 - acc: 0.7876 - val_loss: 0.4431 - val_acc: 0.8711


## 5. Evalue le modèle

In [15]:
# Final evaluation of the model
score = model.evaluate(X_test, y_test, verbose=1)

print('test loss:', score[0])
print('test acc:', score[1])

10000/10000 [==============================] - 1s 111us/step
test loss: 0.4431277368068695
test acc: 0.8710999856948852


In [16]:
score

[0.4431277368068695, 0.8710999856948852]